In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.svm import SVC
import xgboost as xgb
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os
import random
import torch
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from tqdm import tqdm


In [3]:
def seed_everything(seed):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    os.environ['PYTHONHASHSEED']=str(seed)

seed_everything(42)

In [4]:
os.chdir('C:\\Users\\Daniel\\Desktop\\Grad school\\UWaterloo Masters\\Winter 2024\\STAT923\\Final Project\\STAT932_Final_Project\\Images')
original_data = pd.read_csv('data.csv')
original_data.head()

,images,labels
0,20130529_01_Driv_001_f .jpg,0
1,20130529_01_Driv_002_f .jpg,0
2,20130529_01_Driv_003_f .jpg,0
3,20130529_01_Driv_004_f .jpg,0
4,20130529_01_Driv_005_f .jpg,0


In [11]:
os.chdir('C:\\Users\\Daniel\\Desktop\\Grad school\\UWaterloo Masters\\Winter 2024\\STAT923\\Final Project\\STAT932_Final_Project\\GAN\\GAN_generated_images')
gan_data = pd.read_csv('gan_image_df.csv')
gan_data = gan_data.rename(columns = {
    'img_path': "images", 
    'label': 'labels'
})

gan_data.head()

,images,labels
0,C:\Users\Daniel\Desktop\Grad school\UWaterloo ...,2
1,C:\Users\Daniel\Desktop\Grad school\UWaterloo ...,2
2,C:\Users\Daniel\Desktop\Grad school\UWaterloo ...,1
3,C:\Users\Daniel\Desktop\Grad school\UWaterloo ...,2
4,C:\Users\Daniel\Desktop\Grad school\UWaterloo ...,1


We can combine the two dataframes together

In [12]:
result_df = pd.concat([original_data, gan_data])
print(result_df)

                                                images  labels
0                          20130529_01_Driv_001_f .jpg       0
1                          20130529_01_Driv_002_f .jpg       0
2                          20130529_01_Driv_003_f .jpg       0
3                          20130529_01_Driv_004_f .jpg       0
4                          20130529_01_Driv_005_f .jpg       0
..                                                 ...     ...
795  C:\Users\Daniel\Desktop\Grad school\UWaterloo ...       2
796  C:\Users\Daniel\Desktop\Grad school\UWaterloo ...       1
797  C:\Users\Daniel\Desktop\Grad school\UWaterloo ...       2
798  C:\Users\Daniel\Desktop\Grad school\UWaterloo ...       1
799  C:\Users\Daniel\Desktop\Grad school\UWaterloo ...       2

[1406 rows x 2 columns]


In [14]:
X_train, X_test, y_train, y_test = train_test_split(result_df ['images'], result_df ['labels'], 
                                                    test_size = 0.2, stratify= result_df ['labels'],
                                                    random_state=42)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(1124,)
(1124,)
(282,)
(282,)


In [15]:
def preproc_images(image_paths, label_images, target_size = (80,80)):
    images = []
    labels = []
    idx = 0
    for img_path in image_paths:
        with Image.open(img_path) as img:
            img_resized = img.resize(target_size)
            #img_gray = img_resized.convert('L')
            img_array = np.array(img_resized) / 255.0
            images.append(img_array)
            labels.append(label_images[idx])
        idx += 1
    images = np.array(images)
    labels = np.array(labels)
    return images, labels


In [16]:
os.chdir('C:\\Users\\Daniel\\Desktop\\Grad school\\UWaterloo Masters\\Winter 2024\\STAT923\\Final Project\\STAT932_Final_Project\\Images\\DrivImages')
train_images, train_labels = preproc_images(list(X_train), list(y_train))
test_images, test_labels = preproc_images(list(X_test), list(y_test))

We can test it with the best performing ML method as well as the best performing DL method

In [19]:
train_images_reshaped = train_images.reshape(train_images.shape[0], -1)
test_images_reshaped = test_images.reshape(test_images.shape[0], -1)

In [20]:
print(train_images_reshaped.shape)
print(test_images_reshaped.shape)

(1124, 19200)
(282, 19200)


In [22]:
xgb_classifier = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_classifier.fit(train_images_reshaped, train_labels)

y_pred_train_xgb = xgb_classifier.predict(train_images_reshaped)
y_pred_test_xgb = xgb_classifier.predict(test_images_reshaped)

train_acc = accuracy_score(train_labels,y_pred_train_xgb)
test_acc = accuracy_score(test_labels,y_pred_test_xgb)


print(f"Train Accuracy: {train_acc*100:.2f}%")
print(f"Test Accuracy: {test_acc*100:.2f}%")

Train Accuracy: 100.00%
Test Accuracy: 83.69%


In [23]:
os.environ['CUDA'] = '1'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [29]:
class ImageDataset(Dataset):
    def __init__(self, image_data, label_data, transform=None):
        self.image_data = image_data
        self.label_data = label_data
        self.transform = transform

    def __len__(self):
        return len(self.image_data)

    def __getitem__(self, idx):
        feature_row = self.image_data[idx,]
        label_row = self.label_data[idx,]

        h = 80
        w = 80
        num_channels = 3
    
        image  = torch.tensor(feature_row, dtype = torch.float)
        label = torch.tensor(label_row, dtype = torch.long)
        image = image.reshape(num_channels, h,w)

        if self.transform:
            image = self.transform(image)

        return image, label

In [56]:
class MLPDataset(Dataset):
    def __init__(self, image_data, label_data, transform=None):
        self.image_data = image_data
        self.label_data = label_data
        self.transform = transform

    def __len__(self):
        return len(self.image_data)

    def __getitem__(self, idx):
        feature_row = self.image_data[idx,]
        label_row = self.label_data[idx,]

        image  = torch.tensor(feature_row, dtype = torch.float)
        label = torch.tensor(label_row, dtype = torch.long)

        if self.transform:
            image = self.transform(image)

        return image, label

In [71]:
train_dataset = ImageDataset(image_data= train_images_reshaped, label_data = train_labels)
test_dataset = ImageDataset(image_data = test_images_reshaped, label_data = test_labels)

train_dataset_mlp = MLPDataset(image_data= train_images_reshaped, label_data = train_labels)
test_dataset_mlp = MLPDataset(image_data = test_images_reshaped, label_data = test_labels)


train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

train_loader_mlp = DataLoader(train_dataset_mlp, batch_size=8, shuffle=True)
test_loader_mlp = DataLoader(test_dataset_mlp, batch_size=8, shuffle=False)

In [72]:
class EarlyStopping:
    def __init__(self, patience=3, verbose=False, delta=0, path='checkpoint.pt'):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = float('inf')
        self.delta = delta
        self.path = path

    def __call__(self, val_loss, model):
        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decreases.'''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}). Saving model...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [73]:
class MLP(nn.Module):
    def __init__(self, input_size = 19200, hidden_sizes =[1024, 512, 256], num_classes = 3, dropout_rate=0.15):

        super(MLP, self).__init__()
        self.hidden_layers = nn.ModuleList([nn.Linear(input_size, hidden_sizes[0])])
        layer_sizes = zip(hidden_sizes[:-1], hidden_sizes[1:])
        self.hidden_layers.extend([nn.Linear(h1, h2) for h1, h2 in layer_sizes])

        self.output_layer = nn.Linear(hidden_sizes[-1], num_classes)
        
        self.relu = nn.ReLU()

        self.dropout = nn.Dropout(dropout_rate)


    def forward(self, x):
        for layer in self.hidden_layers:
            x = self.relu(layer(x))
            x = self.dropout(x)
        x = self.output_layer(x)
        return x

In [74]:
model_mlp = MLP()
model_mlp.to(device)
optimizer_mlp = torch.optim.Adam(model_mlp.parameters(), lr=0.0005)
criterion = nn.CrossEntropyLoss()

In [76]:
mlp_wts_path = "C:\\Users\\Daniel\\Desktop\\Grad school\\UWaterloo Masters\\Winter 2024\\STAT923\\Final Project\\STAT932_Final_Project\\GAN\\MLP_GAN_checkpoint.pt"
early_stopping = EarlyStopping(patience=5, verbose=True, path = mlp_wts_path)

num_epochs = 100
for epoch in range(100):
    model_mlp.train()
    total_loss = 0.0
    test_loss = 0.0

    for image, labels in tqdm(train_loader_mlp):
        image = image.cuda().to(device)
        labels = labels.cuda().to(device)
        
        optimizer_mlp.zero_grad()

        outputs = model_mlp(image)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer_mlp.step()
        total_loss += loss.item()

    model_mlp.eval()

    with torch.no_grad():
        for images, labels in test_loader_mlp:
            images = images.cuda().to(device)
            labels = labels.cuda().to(device)

            outputs = model_mlp(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
        
    test_loss /= len(test_loader_mlp)
    early_stopping(test_loss, model_mlp)
    if early_stopping.early_stop:
        print("Early stopping")
        break

    print(f'Epoch [{epoch+1}/{100}], Loss: {total_loss/len(train_loader_mlp):.4f}')

model_mlp.load_state_dict(torch.load(mlp_wts_path))

100%|██████████| 141/141 [00:00<00:00, 149.72it/s]


Validation loss decreased (inf --> 0.566082). Saving model...
Epoch [1/100], Loss: 0.5741


100%|██████████| 141/141 [00:00<00:00, 183.32it/s]


Validation loss decreased (0.566082 --> 0.523465). Saving model...
Epoch [2/100], Loss: 0.5622


100%|██████████| 141/141 [00:00<00:00, 186.43it/s]


...

100%|██████████| 141/141 [00:00<00:00, 185.10it/s]


EarlyStopping counter: 4 out of 5
Epoch [21/100], Loss: 0.5220


100%|██████████| 141/141 [00:00<00:00, 184.04it/s]


EarlyStopping counter: 5 out of 5
Early stopping


<All keys matched successfully>

In [77]:
model_mlp.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader_mlp:
        images = images.cuda().to(device)
        labels = labels.cuda().to(device)

        outputs = model_mlp(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy of the network on the test images: {100 * correct / total}%')

Accuracy of the network on the test images: 70.92198581560284%


In [31]:
class CNN(nn.Module):
    def __init__(self, num_classes=3):
        super(CNN, self).__init__()
        
        # Convolutional layer block 1
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        
        # Convolutional layer block 2
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        
        # Convolutional layer block 3
        self.conv3 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        
         # Convolutional layer block 4
        self.conv4 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(512)
        
        # Convolutional layer block 5
        self.conv5 = nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=3, padding=1)
        self.bn5 = nn.BatchNorm2d(1024)
        
        # Max pooling
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Fully connected layers
        self.fc1 = nn.Linear(1024 * 2 * 2,1024) 
        self.fc2 = nn.Linear(1024, num_classes)
        
        # Dropout layer
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = self.pool(F.relu(self.bn4(self.conv4(x))))
        x = self.pool(F.relu(self.bn5(self.conv5(x))))

        x = x.view(-1,  1024 * 2 * 2)

        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        return x

In [33]:
model = CNN()
model.to(device)

CNN(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn4): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv5): Conv2d(512, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn5): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=4096, out_features=1024, bias=True)
  (f

In [34]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) 

In [35]:
cnn_wts_path = "C:\\Users\\Daniel\\Desktop\\Grad school\\UWaterloo Masters\\Winter 2024\\STAT923\\Final Project\\STAT932_Final_Project\\GAN\\CNN_GAN_checkpoint_saved.pt"
early_stopping = EarlyStopping(patience=10, verbose=True, path = cnn_wts_path)

num_epochs = 100
for epoch in range(100):
    model.train()
    total_loss = 0.0
    test_loss = 0.0

    for image, labels in tqdm(train_loader):
        image = image.cuda().to(device)
        labels = labels.cuda().to(device)

        optimizer.zero_grad()

        outputs = model(image) 
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    model.eval()
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.cuda().to(device)
            labels = labels.cuda().to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
        
    test_loss /= len(test_loader)
    early_stopping(test_loss, model)
    if early_stopping.early_stop:
        print("Early stopping")
        break

    print(f'Epoch [{epoch+1}/{100}], Loss: {total_loss/len(train_loader):.4f}')

100%|██████████| 141/141 [00:01<00:00, 76.43it/s] 


Validation loss decreased (inf --> 0.546365). Saving model...
Epoch [1/100], Loss: 0.8597


100%|██████████| 141/141 [00:00<00:00, 148.67it/s]


Validation loss decreased (0.546365 --> 0.533169). Saving model...
Epoch [2/100], Loss: 0.5924


100%|██████████| 141/141 [00:00<00:00, 148.75it/s]


...

EarlyStopping counter: 8 out of 10
Epoch [42/100], Loss: 0.4345


100%|██████████| 141/141 [00:01<00:00, 137.30it/s]


EarlyStopping counter: 9 out of 10
Epoch [43/100], Loss: 0.4261


100%|██████████| 141/141 [00:01<00:00, 137.73it/s]

EarlyStopping counter: 10 out of 10
Early stopping


In [38]:
cnn_best_path = "C:\\Users\\Daniel\\Desktop\\Grad school\\UWaterloo Masters\\Winter 2024\\STAT923\\Final Project\\STAT932_Final_Project\\GAN\\CNN_GAN_checkpoint_saved.pt"
model.load_state_dict(torch.load(cnn_best_path))

<All keys matched successfully>

In [39]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.cuda().to(device)
        labels = labels.cuda().to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy of the network on the test images: {100 * correct / total}%')

Accuracy of the network on the test images: 71.98581560283688%


In [45]:
model2 = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
model2.to(device)
optimizer2 = torch.optim.Adam(model2.parameters(), lr=0.001) 

Using cache found in C:\Users\Daniel/.cache\torch\hub\pytorch_vision_v0.10.0


In [46]:
res_wts_path = "C:\\Users\\Daniel\\Desktop\\Grad school\\UWaterloo Masters\\Winter 2024\\STAT923\\Final Project\\STAT932_Final_Project\\GAN\\Resnet_checkpoint_saved.pt"
early_stopping = EarlyStopping(patience=10, verbose=True, path = res_wts_path)

num_epochs = 100
for epoch in range(100):
    model2.train()
    total_loss = 0.0
    test_loss = 0.0

    for image, labels in tqdm(train_loader):
        image = image.cuda().to(device)
        labels = labels.cuda().to(device)

        optimizer2.zero_grad()

        outputs = model2(image) 
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer2.step()
        total_loss += loss.item()

    model2.eval()
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.cuda().to(device)
            labels = labels.cuda().to(device)

            outputs = model2(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
        
    test_loss /= len(test_loader)
    early_stopping(test_loss, model2)
    if early_stopping.early_stop:
        print("Early stopping")
        break

    print(f'Epoch [{epoch+1}/{100}], Loss: {total_loss/len(train_loader):.4f}')

100%|██████████| 141/141 [00:01<00:00, 97.12it/s] 


Validation loss decreased (inf --> 0.676532). Saving model...
Epoch [1/100], Loss: 1.1313


100%|██████████| 141/141 [00:01<00:00, 109.93it/s]


EarlyStopping counter: 1 out of 10
Epoch [2/100], Loss: 0.6189


100%|██████████| 141/141 [00:01<00:00, 112.07it/s]


...

EarlyStopping counter: 8 out of 10
Epoch [29/100], Loss: 0.1087


100%|██████████| 141/141 [00:01<00:00, 107.39it/s]


EarlyStopping counter: 9 out of 10
Epoch [30/100], Loss: 0.0801


100%|██████████| 141/141 [00:01<00:00, 109.71it/s]


EarlyStopping counter: 10 out of 10
Early stopping


In [47]:
model2.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.cuda().to(device)
        labels = labels.cuda().to(device)

        outputs = model2(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy of the network on the test images: {100 * correct / total}%')

Accuracy of the network on the test images: 81.91489361702128%
